In [ ]:
import sys
from pathlib import Path

# Add parent directory to Python path
parent_dir = Path().resolve().parent
sys.path.insert(0, str(parent_dir))

In [ ]:
%%capture
import logging
from io import BytesIO
from typing import Self, Optional
from datetime import datetime
import pyspark.pandas as pd
from src.spark_session import *

logging.getLogger("pyspark").setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)

In [ ]:
spark = create_spark_session('neo')
spark.sparkContext.setLogLevel('ERROR')

# Get Data

In [ ]:
%%capture
df = spark.sql("SELECT * FROM iceberg_catalog.neo_db.gold_asteroids").toPandas()

# Configure Plots

In [ ]:
import pandas as pd
import panel as pn
import hvplot.pandas


date_slider = pn.widgets.DateSlider(
    name='Select Date',
    start=df['observation_date'].min(),
    end=df['observation_date'].max(),
    value=df['observation_date'].min()
)


def miss_plot(selected_date):
    selected_data = df[df['observation_date'] == selected_date]
    print(selected_date)
    print(type(selected_date))
    return selected_data.hvplot.bar(
    x='name', y='miss_distance_ld',
    title='Miss Distance per Asteroid',
    color='orange', height=300, width=800
    )

def vel_hist(selected_date):
    selected_data = df[df['observation_date'] == selected_date]
    return selected_data.hvplot.hist(
        'velocity_kps', bins=10, title='Velocity Distribution', 
        height=300, width=800)
    

In [ ]:
pn.Column(
    "# 🚀 Near Earth Objects Dashboard",
    date_slider,
    pn.bind(miss_plot,date_slider),
    pn.bind(vel_hist,date_slider)
)

# Dashboard

In [ ]:
close_spark_session(spark)